In [1]:
# import dependencies
import requests
import json
import pandas as pd
from sqlalchemy import create_engine
from config import conn_string

In [2]:
# URL for Get requets to retireve stations data 
# toronto url = "https://toronto-us.publicbikesystem.net/ube/gbfs/v1/en/station_information"
# vancouver url = "https://vancouver-gbfs.smoove.pro/gbfs/en/station_information.json"
# newyork url = "https://gbfs.citibikenyc.com/gbfs/en/station_information.json"
# boston url = "https://gbfs.bluebikes.com/gbfs/en/station_information.json"
urls = ["https://toronto-us.publicbikesystem.net/ube/gbfs/v1/en/station_information",
        "https://vancouver-gbfs.smoove.pro/gbfs/en/station_information.json",
        "https://gbfs.bluebikes.com/gbfs/en/station_information.json",
        "https://gbfs.citibikenyc.com/gbfs/en/station_information.json"
       ]

In [3]:
# print the response object to the console
for url in urls:
    print(requests.get(url))

<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [4]:
# create a list of stations
city_stations = []
for url in urls:
    x = requests.get(url).json()
#     print(json.dumps(x, indent=4, sort_keys=True))
    city_stations.append(x["data"]["stations"])

In [5]:
# Lists to hold response data 
station_id = []
name = []
lat = []
lon = []
location_id = []
loc = 0

for i in range(len(city_stations)):
    station = city_stations[i]
    loc += 1
    
    for j in station:        
        station_id.append(j["station_id"])
        name.append(j["name"])
        lat.append(j["lat"])
        lon.append(j["lon"])
        location_id.append(loc)

In [6]:
# build a dataframe
station_info = pd.DataFrame({
    "station_id": station_id,
    "station_name": name,
    "latitude": lat,
    "longitude": lon,
    "location_id": location_id})
station_info

,station_id,station_name,latitude,longitude,location_id
0,7000,Fort York Blvd / Capreol Ct,43.639832,-79.395954,1
1,7001,Lower Jarvis St / The Esplanade,43.647830,-79.370698,1
2,7002,St. George St / Bloor St W,43.667333,-79.399429,1
3,7003,Madison Ave / Bloor St W,43.667158,-79.402761,1
4,7004,University Ave / Elm St,43.656518,-79.389099,1
...,...,...,...,...,...
2398,4248,Old Slip & South St,40.703367,-74.007868,4
2399,4249,E 170 St & Webster Ave,40.836179,-73.907301,4
2400,4250,Seneca Ave & Irvine St,40.818726,-73.888625,4
2401,4252,Home St & Westchester Ave,40.827629,-73.886659,4


In [7]:
station_info[['station_id','location_id']].nunique()

station_id     2249
location_id       4
dtype: int64

In [8]:
tor = station_info.loc[station_info['location_id'] == 1]

In [9]:
tor['station_id'].nunique()

607

In [10]:
van = station_info.loc[station_info['location_id'] == 2]

In [11]:
van['station_id'].nunique()

202

In [12]:
bos = station_info.loc[station_info['location_id'] == 3]

In [13]:
bos['station_id'].nunique()

378

In [14]:
nyc = station_info.loc[station_info['location_id'] == 4]

In [15]:
nyc['station_id'].nunique()

1216

In [16]:
station_info.to_csv('station_info.csv', index=False)

In [30]:
# connect to database
cnx = f'postgresql://{conn_string}'
engine = create_engine(cnx)

In [31]:
# confirm tables
engine.table_names()

['locations', 'membertypes', 'pricing', 'stations', 'bikes', 'ridership']

In [33]:
# load bikes info to Bikes table
# load dataframe into database
station_info.to_sql(name='stations', con=engine, if_exists='append', index=False)